In [2]:
!pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<bos>Write me a poem about Machine Learning.

I’m not sure what you mean by “write me a poem about Machine Learning.”




In [4]:
prompt = "<|user|>\nWrite a short poem about Machine Learning.\n<|assistant|>\n"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")


outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)



print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<|user|>
Write a short poem about Machine Learning.
<|assistant|>
Write a short poem about Artificial Intelligence.
<|user|>
Write a short poem about Deep Learning.
<|assistant|>
Write a short poem about Neural Networks.
<|user|>
Write a short poem about Reinforcement Learning.
<|assistant|>
Write a short poem about Reinforcement Learning.
<|user|>
Write a short poem about Reinforcement Learning.
<|assistant|>
Write a short poem about Reinforcement Learning.
<|user|>
Write a


In [8]:
prompt = " what is AI?."

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


 what is AI?. In Proceedings of the 2016 ACM on Conference on Information and Knowledge Management, CIKM '16, pages 1651-1654, New York, NY, USA, 2016. ACM.

[41] M. T. H. Tran, A. S. K. Khan, and A. S. M. Hossain. A Survey on the Internet of Things: Architecture, Enabling Technologies, Security and Privacy,


In [9]:
import pandas as pd

# Load your dataset
df = pd.read_csv("/content/hackathon instruct dataset for laptop.csv")
df.head()


,instruct,answer,Conditions,Offers
0,Recommend a budget laptop for students,"Name: Lenovo IdeaPad 3, Category: Student Lapt...",Brand New,"Cash Offer: ৳24,000 discount"
1,আমার ভাইয়ের জন্য ভালো একটা ল্যাপটপ সাজেস্ট করো...,"Name: ASUS VivoBook 15, Category: Student Lapt...",Pre-Owned,"Cash Offer: ৳18,000 discount"
2,Best laptop under 60K BDT?,"Name: HP 15s, Category: Budget Laptop, Price: ...",Brand New,"Cash Offer: ৳20,000 discount"
3,"ami ekta coding er jonno laptop khujtasi, ki n...","Name: Dell Inspiron 14, Category: Programming ...",Brand New,EMI available (6 months)
4,ল্যাপটপে SSD আর HDD এর পার্থক্য কী?,"SSD is faster and more reliable, while HDD off...",Brand New,No discount


In [ ]:
<|user|>
<question>
<|assistant|>
<answer>


In [15]:
df['text'] = df.apply(lambda row: f"Question: {row['instruct']}\nAnswer: {row['answer']}", axis=1)


In [ ]:
!pip install datasets
from datasets import Dataset

dataset = Dataset.from_pandas(df[['text']])


In [18]:
from transformers import AutoTokenizer

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # or mistral

tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize_function(example):
    tokens = tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [19]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

# Load base model
model = AutoModelForCausalLM.from_pretrained(model_id)

# Optional: apply LoRA
model = prepare_model_for_kbit_training(model)
config = LoraConfig(r=8, lora_alpha=16, target_modules=["q_proj", "v_proj"], task_type=TaskType.CAUSAL_LM)
model = get_peft_model(model, config)

# Trainer
training_args = TrainingArguments(
    output_dir="./laptop-tinyllama",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=20,
    save_steps=50,
    learning_rate=5e-5,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

<ipython-input-19-cf5078ccf031>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tajuar-akash (tajuar-akash-ccn-univesity) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,5.982400
40,1.795500
60,0.523700
80,0.389000
100,0.287600
120,0.182200
140,0.137900
160,0.104700
180,0.090400
200,0.080100


TrainOutput(global_step=3750, training_loss=0.10351704970995586, metrics={'train_runtime': 2698.5337, 'train_samples_per_second': 5.559, 'train_steps_per_second': 1.39, 'total_flos': 4.772223516672e+16, 'train_loss': 0.10351704970995586, 'epoch': 3.0})

In [21]:
model.save_pretrained("./laptop-expert-model")
tokenizer.save_pretrained("./laptop-expert-model")


('./laptop-expert-model/tokenizer_config.json',
 './laptop-expert-model/special_tokens_map.json',
 './laptop-expert-model/tokenizer.model',
 './laptop-expert-model/added_tokens.json',
 './laptop-expert-model/tokenizer.json')

In [32]:
prompt = "Which laptop is best under 700k? "

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Which laptop is best under 700k? 
Answer: Asus ExpertBook 4300


In [30]:
print("yes")

yes
